# **Laboratorio RAG - Salud Mental Universitaria**

Este notebook centraliza los pasos del pipeline: ingestión, indexación, recuperación, evaluación y demo RAG. Se utiliza como espacio de experimentación para documentar casos, tomar capturas y depurar.

> Simón Correa Marín

## **Configuración del entorno**
Se cargan las variables del `.env`, detecta la raíz del proyecto y asegura que todas las rutas relativas funcionen tanto desde VS Code como desde el notebook.

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()
NOTEBOOK_DIR = Path.cwd().resolve()
if (NOTEBOOK_DIR / "src").exists():
    PROJECT_ROOT = NOTEBOOK_DIR
else:
    PROJECT_ROOT = NOTEBOOK_DIR.parent
os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
print("Project root:", PROJECT_ROOT)
print("Working dir:", Path.cwd())


Project root: /Users/simon/Documents/rag-salud-mental-universidad
Working dir: /Users/simon/Documents/rag-salud-mental-universidad


## **Revisión rápida de los documentos**
Se inspecciona la carpeta `documentos/`, se toma un archivo de ejemplo y se generan los primeros chunks para verificar que el proceso de ingestión y tokenización responden de manera esperada.

In [2]:
from src.clases_textos import collect_files, extract_text_by_ext, chunk_text, ChunkConfig
docs = collect_files()
print(f'Se encontraron {len(docs)} documentos en la carpeta documentos/')
if docs:
    sample_path = docs[0]
    text = extract_text_by_ext(sample_path)
    config = ChunkConfig(chunk_size=300, overlap=50)
    sample_chunks = chunk_text(text, config.chunk_size, config.overlap)
    print('Documento de ejemplo:', sample_path)
    print('Primer chunk:', sample_chunks[0][:400])


Se encontraron 16 documentos en la carpeta documentos/
Documento de ejemplo: documentos/01.pdf
Primer chunk: ÚLTIMA DÉCADA, N°64, MAYO 2025, PP. 287-315 
https://doi.org/10.5354/0718-2236.2025.78774 
 
 
SALUD MENTAL EN LOS JÓVENES UNIVERSITARIOS: 
UNA MIRADA DESDE LOS PROPIOS ESTUDIANTES 
 
FRANCISCA MENDOZA VILLAGRÁN1 
FRANCISCA PEREIRA CASTRO2 
JORGE YEBER MONTT3 
VERENISSE MOLINA VARGAS4 
FÉLIX COV A SOLAR5 
 
RESUMEN 
Diversos indicadores sugieren un incremento del malestar emocional en los jóvenes.


## **Indexación y generación de artefactos**
Ejecuta el script principal que crea los embeddings con Azure OpenAI para cada configuración (`base`, `small`, `overlap`) y guarda tanto los `.parquet` como los índices FAISS.

In [3]:
# Ejecuta la indexación para regenerar índices y artefactos
!python src/indexacion.py


Procesando configuración 'base' (300 tokens, overlap 50)
  - 01.pdf: 68 chunks
  - 02.pdf: 135 chunks
  - 03.pdf: 67 chunks
  - 04.pdf: 36 chunks
  - 05.pdf: 279 chunks
  - 06.pdf: 51 chunks
  - 07.pdf: 28 chunks
  - 08.pdf: 68 chunks
  - 09.pdf: 40 chunks
  - 10.pdf: 99 chunks
  - 11.pdf: 115 chunks
  - 12.pdf: 44 chunks
  - 13.pdf: 39 chunks
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
  - 14.pdf: 34 chunks
  - 15.pdf: 223 chunks
  - 16.pdf: 42 chunks
  -> Guardado 1368 chunks en data/chunks_base.parquet
  -> Índice FAISS almacenado en indices/faiss_index_base.faiss

Procesando configuración 'small' (200 tokens, overlap 40)
  - 01.pdf: 105 chunks
  - 02.pdf: 211 chunks
  - 03.pdf: 104 chunks
  - 04.pdf: 56 chunks
  - 05.pdf: 436 chunks
  - 06.pdf: 79 chunks
  - 07.pdf: 44 chunks
  - 08.pdf: 106 chunks
  - 09.pdf: 62 chunks
  - 10.pdf: 154 chunk

## **Comparación de recuperaciones**
Antes de pasar a la evaluación global, se consulta manualmente una pregunta y los `top-k` tanto del modelo estándar como del flujo con MMR para validar qué documentos trae cada uno.

In [6]:
# Ejecuta una consulta de ejemplo
!python src/consulta_rag.py


--- Chunks seleccionados ---
[1] doc=02.pdf | cos_sim=0.736
los niños y adolescentes, pues la ansiedad generada por el 
aislamiento, la falta de contacto con los compañeros y las pocas posibilidades de tener espacios 
reguladores de estrés son los principales factores afectantes para esta población. 
Con este panorama, es esencial pensar en discutir las diversas situaciones que pueden 
afectar o impedir el bienestar integral de los estudiantes universitar 

[2] doc=13.pdf | cos_sim=0.733
, donde posiblemente haya 
más rigidez, lleva consigo a una 
vulnerabilidad mayor a padecer trastornos 
de ansiedad (Córdoba, 2020). 
Por consiguiente, este artículo busca situar 
la ansiedad en un punto donde no solo se 
sepa de qué se trata, si no que a través de 
conceptos claros, de cómo ciertos factores 
pueden llevar a un individuo que se 
encuentra en su época universitaria, a ser 
más susc 

[3] doc=09.pdf | cos_sim=0.674
% and for depression from 9.6 to 74.4%. For both 
mental disorders, risk

## **Evaluación**
Se ejecuta `run_experiments()` para calcular `recall@5` y el tamaño medio del contexto en las cuatro variantes del pipeline.

In [7]:
from src.recuperacion_consulta_faiss import search, search_mmr
query = '¿Qué estrategias fortalecen la salud mental estudiantil?'
df_base = search(query, model_name='base', k=5)
df_mmr = search_mmr(query, model_name='overlap', k=5, lambda_mult=0.6)
display(df_base[['doc_id', 'cosine_sim']])
display(df_mmr[['doc_id', 'cosine_sim']])

,doc_id,cosine_sim
0,06.pdf,0.785438
1,08.pdf,0.747695
2,08.pdf,0.741101
3,08.pdf,0.740592
4,08.pdf,0.734437


,doc_id,cosine_sim
0,08.pdf,0.754522
1,14.pdf,0.751094
2,08.pdf,0.743200
3,06.pdf,0.739796
4,06.pdf,0.751626


### **Interpretación de los resultados**
Los valores calculados mostraron que el modelo `mmr` obtuvo el mejor `recall@5` (0.9565) con un contexto medio cercano a 1,484 tokens, lo que indica que incorporó diversidad sin inflar significativamente la ventana. `base` y `small` empataron en `recall@5 = 0.9130`, aunque `small` logró un contexto más compacto (≈1,000 tokens). El modelo `overlap` añadió redundancia sin mejoras claras, por lo que se mantiene como línea base para contrastar las ventajas de MMR.

## **Demo RAG con contexto**
Finalmente se ejecuta `generate_answer()` para obtener la pregunta, los chunks seleccionados y la respuesta del LLM. Esta celda es útil para capturar evidencias visuales y refinar prompts.

In [9]:
from src.evaluation import run_experiments
results = run_experiments()
results

Evaluando modelo 'base'
Evaluando modelo 'small'
Evaluando modelo 'overlap'
Evaluando modelo 'mmr'


,modelo,recall@5,avg_context_len
0,base,0.913043,1500.000000
1,small,0.913043,1000.000000
2,overlap,0.869565,1500.000000
3,mmr,0.956522,1483.913043


## Preguntas de prueba
Estas son las tres preguntas que uso para validar rápidamente el pipeline antes de ejecutar la demo completa:
1. ¿Qué estrategias fortalecen la salud mental estudiantil en los campus?
2. ¿Qué factores de riesgo se mencionan para el bienestar universitario?
3. ¿Cómo describen los documentos el rol de los servicios psicológicos en la promoción del bienestar emocional?


In [ ]:
from src.consulta_rag import generate_answer
question = '¿Qué factores de riesgo se mencionan para el bienestar universitario?'
generate_answer(question, model='mmr', k=5, show_chunks=True)


## Conclusiones y próximos pasos
- El pipeline RAG queda automatizado de extremo a extremo: ingestión → indexación → recuperación → evaluación → demo.
- MMR es la configuración con mejor equilibrio entre precisión y tamaño de contexto, por lo que se recomienda usarla en el informe final y como base para nuevas iteraciones.
- Siguientes pasos sugeridos: aumentar el set de preguntas de evaluación, ajustar `lambda_mult` para distintos escenarios y recopilar capturas desde esta notebook para documentar los hallazgos en el informe.